# Copy Shared Drive Content to MyDrive

> **Note:** Please read the README.md file for additional information.

This notebook helps you copy content from a shared Google Drive folder to your personal MyDrive folder.

## Step 1: Mount Google Drive

First, we need to allow Colab to access your Google Drive. Run the cell below to mount your drive.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

## Step 2: Verify Drive Access

Let's verify that the drive is mounted correctly by listing the contents of MyDrive (optional).

In [ ]:
!ls -l "/content/drive/MyDrive"

## Step 3: Install Required Libraries

Install the Google Drive API library if not already installed.

In [ ]:
!pip install -q google-api-python-client google-auth-httplib2 google-auth-oauthlib


## Step 4: Copy Folder Content from Google Drive Link

This step will copy the source folder from a Google Drive link to your MyDrive. Enter the Google Drive folder link (URL).


In [ ]:
import re
from googleapiclient.discovery import build
from google.colab import auth

auth.authenticate_user()
drive_service = build('drive', 'v3')

def extract_folder_id(url):
    """Extract folder ID from Google Drive URL"""

    patterns = [
        r'/folders/([a-zA-Z0-9_-]+)',
        r'id=([a-zA-Z0-9_-]+)',
        r'folders/([a-zA-Z0-9_-]+)',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    

    if re.match(r'^[a-zA-Z0-9_-]+$', url):
        return url
    
    raise ValueError("Could not extract folder ID from URL. Please check the link format.")

def get_folder_name(service, folder_id):
    """Get folder name from folder ID"""
    try:
        file_metadata = service.files().get(fileId=folder_id, fields='name').execute()
        return file_metadata.get('name', 'Unknown')
    except Exception as e:
        print(f"Error getting folder name: {e}")
        return 'CopiedFolder'

def copy_folder_recursive(service, source_folder_id, dest_parent_id, dest_folder_name):
    """Recursively copy folder and its contents"""

    folder_metadata = {
        'name': dest_folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [dest_parent_id]
    }
    new_folder = service.files().create(body=folder_metadata, fields='id').execute()
    new_folder_id = new_folder.get('id')
    
    query = f"'{source_folder_id}' in parents and trashed=false"
    results = service.files().list(q=query, fields="files(id, name, mimeType)").execute()
    items = results.get('files', [])
    
    for item in items:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            # Recursively copy subfolder
            copy_folder_recursive(service, item['id'], new_folder_id, item['name'])
        else:
            # Copy file
            try:
                copied_file = service.files().copy(fileId=item['id'], 
                                                   body={'parents': [new_folder_id]}).execute()
            except Exception as e:
                print(f"Warning: Could not copy file '{item['name']}': {e}")
    
    return new_folder_id

# Get inputs
source_link = str(input("Enter the Google Drive folder link (URL): ")).strip()
dest_folder_name = str(input("Enter the name for the destination folder in MyDrive: ")).strip()

try:
    # Extract folder ID from link
    source_folder_id = extract_folder_id(source_link)
    print(f"Extracted folder ID: {source_folder_id}")
    
    # Get source folder name
    source_folder_name = get_folder_name(drive_service, source_folder_id)
    print(f"Source folder: {source_folder_name}")
    
    # Get MyDrive (root) folder ID
    root_folder = drive_service.files().get(fileId='root', fields='id').execute()
    dest_parent_id = root_folder.get('id')
    
    print(f"\nCopying folder '{source_folder_name}' to MyDrive as '{dest_folder_name}'...")
    print("This may take a while depending on the folder size...\n")
    
    # Copy folder recursively
    new_folder_id = copy_folder_recursive(drive_service, source_folder_id, dest_parent_id, dest_folder_name)
    
    print(f"Copy completed successfully!")
    print(f"Folder ID: {new_folder_id}")
    print(f"You can find the copied folder in your MyDrive as '{dest_folder_name}'")
    
except Exception as e:
    print(f"Error during copy: {e}")
    print(f"Error type: {type(e).__name__}")
